imports

In [1]:
import numpy as np
from scipy.io import mmread
from scipy import sparse
import time
import networkx as nx

loading dataset

In [2]:
data = mmread('web-BerkStan.mtx')
data = data.tocsc()[:1000, :1000]
data = data.toarray()

Pagerank algorithm using np.linalg.eig

In [3]:
def pagerank_lib(matrix, alpha):
    n = matrix.shape[0]
    e = np.ones(n)
    d = np.zeros(n)

    # define Q matrix by normalizing input matrix
    Q = matrix
    sum_col = Q.sum(axis=0)

    # d is needed for next parts
    d[np.where(sum_col==0)]=1
    sum_col[np.where(sum_col==0)]=1 # avoiding devision by zero
    Q = Q/sum_col

    # define P matrix(stochastic matrix)
    P = Q + ((e@d.T)/n)
    
    # define A matrix
    A = alpha*P + ((1-alpha)/n)*(e@e.T)
    
    return np.linalg.eig(A).eigenvectors.T[0].real

start = time.time()
r_lib = pagerank_lib(data.T, 0.85)
time_lib = time.time()-start

top_20_indices = np.argsort(r_lib)[-20:][::-1]
top_20_scores = r_lib[top_20_indices]

print("Top 20 Pages:")
for idx, score in zip(top_20_indices, top_20_scores):
    print(f"Page ID: {idx}, PageRank Score: {score}")

Top 20 Pages:
Page ID: 39, PageRank Score: 0.04936581382495595
Page ID: 860, PageRank Score: 0.03811308735747028
Page ID: 767, PageRank Score: 0.037216513683417884
Page ID: 371, PageRank Score: 0.03534615255500336
Page ID: 256, PageRank Score: 0.034851590278168865
Page ID: 23, PageRank Score: 0.034803574893768625
Page ID: 926, PageRank Score: 0.0346072429948062
Page ID: 295, PageRank Score: 0.03446539526053894
Page ID: 509, PageRank Score: 0.03437958689105596
Page ID: 329, PageRank Score: 0.0342994745615091
Page ID: 612, PageRank Score: 0.0341730825089172
Page ID: 692, PageRank Score: 0.03406272072207295
Page ID: 459, PageRank Score: 0.033823284193474634
Page ID: 564, PageRank Score: 0.033820193729866485
Page ID: 425, PageRank Score: 0.03363515946962345
Page ID: 742, PageRank Score: 0.03341637024155727
Page ID: 197, PageRank Score: 0.03339593132340186
Page ID: 32, PageRank Score: 0.03339483436429722
Page ID: 232, PageRank Score: 0.03318074314597347
Page ID: 160, PageRank Score: 0.03317

Pagerank algorithm using Power method

In [4]:
def pagerank_powermethod(matrix, alpha, num_iterations=100):
    n = matrix.shape[0]
    e = np.ones(n)
    
    Q = matrix
    sum_col = Q.sum(axis=0)
    sum_col[np.where(sum_col==0)]=1 # avoiding devision by zero
    Q = Q/sum_col
    
    r = np.ones(n)/n
    #sparse Q
    sQ = sparse.csr_matrix(Q)
    for _ in range(num_iterations):
        beta = 1 - np.linalg.norm(alpha*sQ@r,1)
        r = alpha*sQ@r+(beta/n)*e
    
    return r

start = time.time()
r_power = pagerank_powermethod(data.T, 0.85)
time_power = time.time()-start

top_20_indices = np.argsort(r_power)[-20:][::-1]
top_20_scores = r_power[top_20_indices]

print("Top 20 Pages:")
for idx, score in zip(top_20_indices, top_20_scores):
    print(f"Page ID: {idx}, PageRank Score: {score}")


Top 20 Pages:
Page ID: 860, PageRank Score: 0.037339018285163124
Page ID: 767, PageRank Score: 0.03409353941091894
Page ID: 39, PageRank Score: 0.031264801813871125
Page ID: 926, PageRank Score: 0.028489368991158153
Page ID: 371, PageRank Score: 0.014183179677400126
Page ID: 742, PageRank Score: 0.013386018851682774
Page ID: 934, PageRank Score: 0.01296768097366021
Page ID: 256, PageRank Score: 0.01252323975634228
Page ID: 803, PageRank Score: 0.012284514637246755
Page ID: 32, PageRank Score: 0.01226138945029178
Page ID: 967, PageRank Score: 0.012203665540833951
Page ID: 944, PageRank Score: 0.012033850596198957
Page ID: 714, PageRank Score: 0.01166482102041012
Page ID: 719, PageRank Score: 0.011530591356115392
Page ID: 923, PageRank Score: 0.011530591356115392
Page ID: 720, PageRank Score: 0.011471920283654754
Page ID: 295, PageRank Score: 0.010722576881532343
Page ID: 329, PageRank Score: 0.009994023717161292
Page ID: 812, PageRank Score: 0.007888344980349479
Page ID: 197, PageRank S

Comparing two method based on the execution time and output

In [5]:
Q = data.T
sum_col = Q.sum(axis=0)
sum_col[np.where(sum_col==0)]=1 # avoiding devision by zero
Q = Q/sum_col
    
# comparing outputs
print("measuring accuracy of output based on pagerank using numpy.linalg.eig:", np.linalg.norm(Q@r_lib - r_lib), "measuring accuracy of output based on pagerank using powermethod:", np.linalg.norm(Q@r_power - r_power))

# comparing execution times
print("execution time of pagerank using numpy.linalg.eig:", time_lib, "execution time of pagerank using powermethod:", time_power)

measuring accuracy of output based on pagerank using numpy.linalg.eig: 4.304778950449738 measuring accuracy of output based on pagerank using powermethod: 0.01457799111872476
execution time of pagerank using numpy.linalg.eig: 1.0247840881347656 execution time of pagerank using powermethod: 0.030916452407836914


pagerank using networks

In [6]:
G = nx.to_networkx_graph(data, create_using=nx.DiGraph)
start = time.time()
r_nx = nx.pagerank(G)
time_nx = time.time() - start
top_20_nx = sorted(r_nx.items(), key=lambda x: x[1], reverse=True)[:20]
print("Top 20 Pages from NetworkX:")
for idx, score in top_20_nx:
    print(f"Page ID: {idx}, PageRank Score: {score}")

Top 20 Pages from NetworkX:
Page ID: 860, PageRank Score: 0.036802523198682005
Page ID: 767, PageRank Score: 0.0342310012052098
Page ID: 39, PageRank Score: 0.03127589443265558
Page ID: 926, PageRank Score: 0.028610857570783414
Page ID: 371, PageRank Score: 0.014206404510008304
Page ID: 742, PageRank Score: 0.01343036408888926
Page ID: 934, PageRank Score: 0.013012341041687164
Page ID: 256, PageRank Score: 0.01253523750494919
Page ID: 803, PageRank Score: 0.01232731447875722
Page ID: 32, PageRank Score: 0.012300746888460092
Page ID: 967, PageRank Score: 0.012244346977842891
Page ID: 944, PageRank Score: 0.012074902379336313
Page ID: 714, PageRank Score: 0.01170500738459077
Page ID: 719, PageRank Score: 0.011570219368020974
Page ID: 923, PageRank Score: 0.011570219368020974
Page ID: 720, PageRank Score: 0.011511305095388018
Page ID: 295, PageRank Score: 0.010745637925326897
Page ID: 329, PageRank Score: 0.010016107056598157
Page ID: 812, PageRank Score: 0.007816352397179465
Page ID: 197

compare

In [7]:
r_nx = np.array(list(r_nx.values()))
np.linalg.norm(Q@r_nx-r_nx), time_nx

(0.014501402341095789, 0.013960838317871094)

alpha

In [8]:
alpha_values = [0.25, 0.5, 0.75, 0.99]
for alpha in alpha_values:
    r = pagerank_powermethod(data.T, alpha)
    print(f"PageRank for alpha={alpha}:")
    print(np.linalg.norm(Q@r-r))

PageRank for alpha=0.25:
0.09042993699169666
PageRank for alpha=0.5:
0.05315494038506027
PageRank for alpha=0.75:
0.024096448689294385
PageRank for alpha=0.99:
0.002194506230480889


Q2

In [9]:
M = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
              [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
roads = pagerank_powermethod(M, 0.85)
dict(zip(roads.argsort()[::-1], np.sort(roads)[::-1]))

{5: 0.22178581561416394,
 2: 0.1483512614371394,
 4: 0.13309283268967886,
 9: 0.1293006643686188,
 7: 0.07814005033301585,
 1: 0.07814005033301585,
 10: 0.04223786504487344,
 8: 0.04223786504487344,
 6: 0.04223786504487344,
 3: 0.04223786504487344,
 0: 0.04223786504487344}

In [10]:
H = nx.to_networkx_graph(M.T, create_using=nx.DiGraph)
nx.pagerank(H)

{0: 0.042237632230445084,
 1: 0.07813991093909312,
 2: 0.1483519515396573,
 3: 0.042237632230445084,
 4: 0.1330931108197563,
 5: 0.22178592424253255,
 6: 0.042237632230445084,
 7: 0.07813991093909312,
 8: 0.042237632230445084,
 9: 0.12930103036764215,
 10: 0.042237632230445084}